# 분산 분석
- section 01 : 일원 분산 분석
- section 02 : 이원 분산 분석

## section 01 : 일원 분산 분석
- 3개 이상의 집단 간의 평균 차이가 통계적으로 유의한지 검정하는 방법
- 집단을 나누는 요인이 하나이고 집단의 수가 3개 이상일 때 사용


### 기본 가정
- 분산 분석의 기본 가정은 독립 표본 t-검정과 매우 유사
    - 독립성 : 각 집단의 관측치들은 모든 다른 집단의 관측치들과 독립적
    - 정규성 : 각 집단에서의 관측치는 정규분포를 따른다.
    - 등분산성 : 모든 집단에서의 관측치는 동일한 분산을 가진다.

### 귀무가설과 대립가설
- 귀무가설(H0) : 모든 집단의 평균은 동일
- 대립가설(H1) : 집단의 평균에는 차이가 있다.

### 일원 분산 분석
- 사이파이의 f_oneway를 사용한다.
```python
scipy.stats.f_oneway(sample1,sample2,sample3,...)
``` 
- smaple1,2,3 : 집단 데이터


In [2]:
# 문제
# 데이터는 4종류의 비료를 사용한 식물의 성장에 대한 실험 결과이다.
# 이 실험에서는 비슷한 조건의 식물 40개를 무작위로 10개씩 나누고 화학 비료 A,B,C,D를 일정 기간 사용한 후 성장량을 측정했다.
# 성장의 차이가 있는지 유의수준 0.05하에서 검정

# 귀무가설 : 네 가지 비료의 효과는 동일하다.
# 대립가설 : 비료의 효과에는 차이가 있다.

import pandas as pd

df = pd.DataFrame({
    'A':[10.5,11.3,10.8,9.6,11.1,10.2,10.9,11.4,10.5,10.3],
    'B':[11.9,12.4,12.1,13.2,12.5,11.8,12.2,12.9,12.4,12.3],
    'C':[11.2,11.7,11.6,10.9,11.3,11.1,10.8,11.5,11.4,11.0],
    'D':[9.8,9.4,9.1,9.5,9.6,9.9,9.2,9.7,9.3,9.4]
})

df.head(2)

,A,B,C,D
0,10.5,11.9,11.2,9.8
1,11.3,12.4,11.7,9.4


- 정규성 검정 : A,B,C,D의 모든 그룹에 대한 p-value가 0.05보다 크므로 모든 그룹의 데이터가 정규분포를 따름.
- 등분산성 검정 : p-value가 0.05보다 크므로 모든 그룹의 분산이 동일하다고 볼 수 있다.
- 일원 분산 분석 : p-value가 0.05보다 작으므로 적어도 두 그룹 간의 평균 성장량에는 차이가 있다고 볼 수 있다.

In [3]:
from scipy import stats

print('=== 정규성 검정 ===')
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))

print('\n=== 등분산성 검정 ===')
print(stats.levene(df['A'],df['B'],df['C'],df['D']))

print('\n=== 일원 분산 분석')
print(stats.f_oneway(df['A'],df['B'],df['C'],df['D']))


=== 정규성 검정 ===
ShapiroResult(statistic=np.float64(0.9649054066073819), pvalue=np.float64(0.840016154346871))
ShapiroResult(statistic=np.float64(0.9468040874196029), pvalue=np.float64(0.6308700692815115))
ShapiroResult(statistic=np.float64(0.9701646110856058), pvalue=np.float64(0.8923673061902999))
ShapiroResult(statistic=np.float64(0.9752339025839641), pvalue=np.float64(0.9346854448707635))

=== 등분산성 검정 ===
LeveneResult(statistic=np.float64(1.9355354288758708), pvalue=np.float64(0.14127835331346628))

=== 일원 분산 분석
F_onewayResult(statistic=np.float64(89.12613851177174), pvalue=np.float64(1.001838152252373e-16))


### 일원 분산 분석(ols 활용)
- 일원 분산 분석 방법에는 사이파이의 f_oneway와 스태츠모델즈의 ols,anova_lmd을 사용하는 방법이 있다.
- ols() 함수는 R 스타일의 표현식을 사용한다.


<table>
  <tr>
    <td>구분</td>
    <td>설명</td>
  </tr>
  <tr>
    <td rowspan="2">물결(~)</td>
    <td>종속변수와 독립 변수를 구분하는 기호</td>
    
  </tr>
  <tr>
  <td>target~a에서 target은 종속변수고, a는 독립변수임을 나타낸다.</td>
  </tr>
    <tr>
    <td rowspan="2">더하기(+)</td>
    <td>여러 독립변수들을 모델에 포함하기 위한 기호다.</td>
    
  </tr>
  <tr>
  <td>target~a+b+c+d 는 a,b,c,d 독립변수를 모두 포함하겠다는 의미.</td>
  </tr>
</table>

In [6]:
# !pip install statsmodels
!pip install patsy

In [9]:
import pandas as pd
df = pd.read_csv('./ch2_data/fertilizer.csv')

In [10]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('성장~C(비료)',df).fit()
print(anova_lm(model))

            df    sum_sq    mean_sq          F        PR(>F)
C(비료)      3.0  43.21875  14.406250  89.126139  1.001838e-16
Residual  36.0   5.81900   0.161639        NaN           NaN


<table>
  <tr>
    <td>구분</td>
    <td>설명</td>
  </tr>
  <tr>
    <td rowspan="4">df(자유도)</td>
    <td>전체 데이터 관찰 수 : 4*10 = 40</td>
  </tr>
  <tr>
  <td>전체 데이터(관찰) 수의 자유도 : 40 -1 = 39</td>
  </tr>
    <tr>
  <td>그룹(처리조건)의 자유도: 4 - 1 = 3 </td>
  </tr>
    <tr>
  <td>잔차의 자유도 : 39 -3 = 36</td>
  </tr>
    <tr>
    <td rowspan="1">sum_sq(제곱합)</td>
    <td>그룹(처리조건)별 또는 잔차별 제곱합</td>
  </tr>
   <tr>
    <td rowspan="1">mean_sq(평균 제곱)</td>
    <td>평균 제곱합으로 sum_sq를 df로 나눈 값</td>
  </tr>
  <tr>
    <td rowspan="1">F(F-통계량)</td>
    <td>F-통계량</td>
  </tr>
  <tr>
    <td rowspan="1">PR(>F)(p-값)</td>
    <td>F-통계량에 대한 p-value</td>
  </tr>
</table>

## section 02 : 이원 분산 분석
- 이원 분산 분석(Two-way ANOVA)은 요인의 수가 2개인 경우.
<table>
  <tr>
    <td>A 요인 : 학습방법</td>
    <td colspan="2">도서</td>
    <td colspan="2">강의</td>
  </tr>
  <tr>
    <td>B 요인 : 학습 장소</td>
    <td>집</td>
    <td>카페</td>
    <td>집</td>
    <td>카페</td>
  </tr>
</table>

### 기본 가정
- 독립성 : 각 집단의 관측치들은 모든 다른 집단의 관측치들과 독립적이다.(기본 가정)
- 정규성 : 각 집단에서의 관측치는 정규 분포를 따른다. (Shapiro-Wilk test)
- 등분산성 : 모든 집단에서의 관측치는 동일한 분산을 가진다(Levene test)

### 귀무가설과 대립가설
- 이원 분산 분석은 주 효과뿐만 아니라 상호작용 효과에도 관심을 둔다.
- 학습 방법과 학습 장소를 요인으로 이원 분산 분석을 실시할 경우 주 효과 2개, 상호작용 효과 1개로 다음과 같은 가설을 세울 수 있음.

- 주 효과 : A 요인(학습방법)
    - 귀무가설 : 학습 방법에 따라 성적 차이가 없다.
    - 대립가설 : 학습 방법에 따라 성척 차이가 있다.
- 주 효과 : B 요인(학습장소)
    - 귀무가설 : 학습 장소에 따라 성적 차이가 없다.
    - 대립가설 : 학습 장소에 따라 성척 차이가 있다.

- 상호 작용 효과
    - 귀무가설 : A 요인과 B 요인 간에 따라 상호작용이 없다.
    - 대립가설 : A 요인과 B 요인 간에 따라 상호작용이 있다.
---